In [1]:
import os

In [2]:
%pwd

'c:\\Users\\pc\\Desktop\\ai\\mlops-datascience\\research'

In [3]:
os.chdir("../")

In [4]:
import pandas as pd
data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir:Path
    data_path:Path

In [9]:
from src.mlops_pipeline.constants import *
from src.mlops_pipeline.utils.common import read_yaml,create_directories

In [10]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml([config_filepath])
        self.params = read_yaml([params_filepath])
        self.schema = read_yaml([schema_filepath])
        
        
        create_directories([self.config.artifacts_root])
        
        
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config= self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path = config.data_path
        )
        return data_transformation_config
        

In [11]:
import os
from src.mlops_pipeline import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [12]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config=config
        
    def train_test_splitting(self):
        data = pd.read_csv(self.config.data_path)
        train,test = train_test_split(data)
        train.to_csv(os.path.join(self.config.root_dir,"train.csv"),index=False)
        test.to_csv(os.path.join(self.config.root_dir,"test.csv"),Index=False)
        
        logger.info("Splitted data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)
        print(train.shape)
        print(test.shape)

In [13]:
try:
    config= ConfigurationManager()
    data_transformation_config= config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_splitting()
except Exception as e:
    raise e

EnsureError: Argument path_to_yaml of type <class 'list'> to <function read_yaml at 0x000001ADC22C01F0> does not match annotation type <class 'pathlib.Path'>